In [1]:
# Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
# !pip install joblib

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [12]:
# the dataset used contains all delays under 30 mins. 
df = pd.read_csv("../Data/ttc_subway_delay_2018_2019_for_machine_learning.csv", encoding='unicode_escape')
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
df.head()
# one hot encoding for stations - 1 or 0. will create many columns
# min_delay
# time series model

,id,date,time,day,station,code,min_delay,min_gap,bound,line,vehicle,code_info,latitude,longitude,line_name,month,time_range,month_number,hour,year
0,1,2019-01-01,3:03,Tuesday,DUPONT STATION,MUATC,11,16,N,YU,6061,ATC Project,43.674584,-79.406830,Yonge University Spadina,January,9PM-1:30AM,1,3,2019
1,2,2019-01-01,3:08,Tuesday,EGLINTON WEST STATION,EUATC,11,16,S,YU,5656,ATC RC&S Equipment,43.699209,-79.435819,Yonge University Spadina,January,9PM-1:30AM,1,3,2019
2,3,2019-01-01,3:09,Tuesday,DUPONT STATION,EUATC,6,11,N,YU,5381,ATC RC&S Equipment,43.674584,-79.406830,Yonge University Spadina,January,9PM-1:30AM,1,3,2019
3,4,2019-01-01,3:26,Tuesday,ST CLAIR WEST STATION,EUATC,4,9,N,YU,5571,ATC RC&S Equipment,43.683888,-79.415113,Yonge University Spadina,January,9PM-1:30AM,1,3,2019
4,5,2019-01-01,8:04,Tuesday,DAVISVILLE STATION,MUNOA,5,10,S,YU,0,No Operator Immediately Available - Not E.S.A....,43.697778,-79.397222,Yonge University Spadina,January,5-9AM,1,8,2019


# Select your features (columns)

In [13]:
# Drop the null rows, and everything that is not a float except station
df = df.dropna().drop(["latitude",
                       "longitude",
                       "code_info",
                       "line_name",
                       "month",
                       "time_range",
                       "vehicle"
                      ], axis=1)
df.head()

,id,date,time,day,station,code,min_delay,min_gap,bound,line,month_number,hour,year
0,1,2019-01-01,3:03,Tuesday,DUPONT STATION,MUATC,11,16,N,YU,1,3,2019
1,2,2019-01-01,3:08,Tuesday,EGLINTON WEST STATION,EUATC,11,16,S,YU,1,3,2019
2,3,2019-01-01,3:09,Tuesday,DUPONT STATION,EUATC,6,11,N,YU,1,3,2019
3,4,2019-01-01,3:26,Tuesday,ST CLAIR WEST STATION,EUATC,4,9,N,YU,1,3,2019
4,5,2019-01-01,8:04,Tuesday,DAVISVILLE STATION,MUNOA,5,10,S,YU,1,8,2019


# Create a Train Test Split



In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
X = df.drop("min_delay", axis=1)
y = df["min_delay"]

# X = df.drop("station", axis=1)
# y = df["min_delay"]
print(X.shape, y.shape)

(13320, 12) (13320,)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=42)

In [17]:
X_train.head()

,id,date,time,day,station,code,min_gap,bound,line,month_number,hour,year
9167,9366,2018-05-19,14:09,Saturday,DON MILLS STATION,PUSO,23,W,SHP,5,14,2018
12235,12474,2018-10-18,12:25,Thursday,WILSON STATION,PUTS,16,N,YU,10,12,2018
13216,13472,2018-12-14,7:00,Friday,SUMMERHILL STATION,EUNT,5,N,YU,12,7,2018
5910,6046,2019-11-25,8:14,Monday,YONGE BLOOR YUS STATION,MUI,6,N,YU,11,8,2019
3205,3262,2019-06-20,13:49,Thursday,KEELE STATION,PUSTS,6,E,BD,6,13,2019


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [19]:
# Scale data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

ValueError: could not convert string to float: '2018-05-19'

In [18]:
# Encode Data 
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [108]:
# categorize
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [147]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# create model
model = Sequential()
model.add(Dense(units=10, activation='relu', input_dim=9)) #how many units should we have?
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=29, activation='softmax'))

In [148]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [150]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=30, #did I over train?
    shuffle=True,
    verbose=2
)

Train on 8921 samples
Epoch 1/30
8921/8921 - 0s - loss: 0.3898 - accuracy: 0.9238
Epoch 2/30
8921/8921 - 0s - loss: 0.3666 - accuracy: 0.9298
Epoch 3/30
8921/8921 - 0s - loss: 0.3440 - accuracy: 0.9338
Epoch 4/30
8921/8921 - 0s - loss: 0.3222 - accuracy: 0.9380
Epoch 5/30
8921/8921 - 0s - loss: 0.3038 - accuracy: 0.9407
Epoch 6/30
8921/8921 - 0s - loss: 0.2864 - accuracy: 0.9471
Epoch 7/30
8921/8921 - 0s - loss: 0.2704 - accuracy: 0.9478
Epoch 8/30
8921/8921 - 0s - loss: 0.2562 - accuracy: 0.9539
Epoch 9/30
8921/8921 - 0s - loss: 0.2430 - accuracy: 0.9559
Epoch 10/30
8921/8921 - 0s - loss: 0.2293 - accuracy: 0.9574
Epoch 11/30
8921/8921 - 0s - loss: 0.2188 - accuracy: 0.9595
Epoch 12/30
8921/8921 - 0s - loss: 0.2076 - accuracy: 0.9611
Epoch 13/30
8921/8921 - 0s - loss: 0.1974 - accuracy: 0.9605
Epoch 14/30
8921/8921 - 0s - loss: 0.1894 - accuracy: 0.9639
Epoch 15/30
8921/8921 - 0s - loss: 0.1799 - accuracy: 0.9649
Epoch 16/30
8921/8921 - 0s - loss: 0.1712 - accuracy: 0.9666
Epoch 17/30

In [151]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 10)                100       
_________________________________________________________________
dense_19 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_20 (Dense)             (None, 29)                319       
Total params: 529
Trainable params: 529
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# prediction result



# Train the Model



In [152]:
# Create the SVC Model
from sklearn.svm import SVC 
model2 = SVC(kernel='linear')
model2

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [153]:
# fit the model
model2.fit(X_train_scaled, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [154]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

Training Data Score: 0.7372491873108395
Testing Data Score: 0.7289485662266727


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [155]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.0005, 0.001]}
grid = GridSearchCV(model2, param_grid, verbose=3)

In [156]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=1, gamma=0.0001, score=0.719, total=   2.3s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.724, total=   2.1s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.3s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.732, total=   2.1s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.729, total=   2.1s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.724, total=   2.1s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.719, total=   2.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.724, total=   2.1s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.732, total=   2.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ................... C=1, gamma=0.0005, score=0.729, total=   2.1s
[CV] C=1, gamma=0.0005 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.0005, 0.001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [157]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'gamma': 0.0001}
0.9641292032508071


# Save the Model

In [158]:
# using results above to save the best model
best_model=SVC(kernel='linear', C=10, gamma=0.0001)

import joblib

filename = 'best_model.sav'
joblib.dump(best_model, filename)

['best_model.sav']